In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import geojson
import rasterio
import pygmsh
import firedrake
from firedrake import sqrt, inner, grad, dx
import icepack
from icepack.constants import (
    ice_density as ρ_I, gravity as g, weertman_sliding_law as m
)

Load up the hand-digitized outline of the glacier, convert it into the input format for gmsh, generate the mesh using gmsh, and then load up the triangular mesh.

In [ ]:
outline_filename = "beardmore.geojson"
with open(outline_filename, "r") as outline_file:
    outline = geojson.load(outline_file)

geometry = icepack.meshing.collection_to_geo(outline)
with open("beardmore.geo", "w") as geo_file:
    geo_file.write(geometry.get_code())
    
command = "gmsh -2 -format msh2 -v 2 -o beardmore.msh beardmore.geo"
subprocess.run(command.split(" "))

mesh = firedrake.Mesh("beardmore.msh")

Fetch the Mosaic of Antarctica so that we can make pretty pictures.

In [ ]:
coords = np.array(list(geojson.utils.coords(outline)))
delta = 10e3
xmin, xmax = coords[:, 0].min() - delta, coords[:, 0].max() + delta
ymin, ymax = coords[:, 1].min() - delta, coords[:, 1].max() + delta

In [ ]:
image_filename = icepack.datasets.fetch_mosaic_of_antarctica()
with rasterio.open(image_filename, "r") as image_file:
    height, width = image_file.height, image_file.width
    transform = image_file.transform
    window = rasterio.windows.from_bounds(
        left=xmin,
        bottom=ymin,
        right=xmax,
        top=ymax,
        width=width,
        height=height,
        transform=transform,
    )
    image = image_file.read(indexes=1, window=window, masked=True)

In [ ]:
def subplots(*args, **kwargs):
    fig, axes = plt.subplots()
    axes.set_aspect("equal")
    xmin, ymin, xmax, ymax = rasterio.windows.bounds(window, transform)
    axes.imshow(
        image,
        cmap="Greys_r",
        vmin=12e3,
        vmax=16.38e3,
        extent=(xmin, xmax, ymin, ymax),
    )

    return fig, axes

Show the mesh on top of some imagery so we can make sure everything is where it should be.

In [ ]:
fig, axes = subplots()
kwargs = {
    "interior_kw": {"linewidth": 0.25},
    "boundary_kw": {"linewidth": 2},
}
firedrake.triplot(mesh, axes=axes, **kwargs)
axes.legend();

Make some function spaces -- here we're deciding to use piecewise quadratic basis functions in each triangle to represent fields defined on this mesh.

In [ ]:
Q = firedrake.FunctionSpace(mesh, family="CG", degree=2)
V = firedrake.VectorFunctionSpace(mesh, family="CG", degree=2)

Start fetching some observational data sets and interpolating them to our finite element spaces.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()
thickness_filename = f"netcdf:{bedmachine_filename}:thickness"
with rasterio.open(thickness_filename, "r") as thickness_file:
    h = icepack.interpolate(thickness_file, Q)
    
surface_filename = f"netcdf:{bedmachine_filename}:surface"
with rasterio.open(surface_filename, "r") as surface_file:
    s = icepack.interpolate(surface_file, Q)

Plot the thickness and surface elevation.

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(s, axes=axes)
fig.colorbar(colors);

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(h, axes=axes)
fig.colorbar(colors);

Now fetch and interpolate the velocities.

In [ ]:
measures_filename = icepack.datasets.fetch_measures_antarctica()
with rasterio.open(f"netcdf:{measures_filename}:VX", "r") as vx_file, \
     rasterio.open(f"netcdf:{measures_filename}:VY", "r") as vy_file:
    u = icepack.interpolate((vx_file, vy_file), V)

In [ ]:
log_norm = matplotlib.colors.LogNorm(vmin=1.0, vmax=400.0)

fig, axes = subplots()
streamlines = firedrake.streamplot(
    u, norm=log_norm, axes=axes, resolution=2.5e3, seed=1729
)
fig.colorbar(streamlines);

Next we'll calculate a very smoothed-over driving stress.
We'll use this to guess at the friction coefficient.

In [ ]:
α = firedrake.Constant(10e3)
τ = firedrake.Function(V)
τ_d = -ρ_I * g * h * grad(s)
misfit = 0.5 * inner(τ - τ_d, τ - τ_d) * dx
smoothness = 0.5 * α ** 2 * inner(grad(τ), grad(τ)) * dx
J = misfit + smoothness
F = firedrake.derivative(J, τ)
firedrake.solve(F == 0, τ)

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(τ, axes=axes)
fig.colorbar(colors);

We'll estimate that basal friction coefficient takes up half the driving stress and that the ice is at a uniform temperature of -13C.

In [ ]:
fraction = firedrake.Constant(0.5)
expr = fraction * sqrt(inner(τ, τ)) / sqrt(inner(u, u)) ** (1 / m)
C = firedrake.interpolate(expr, Q)

In [ ]:
T = firedrake.Constant(260.0)
A = icepack.rate_factor(T)

In [ ]:
model = icepack.models.IceStream()
opts = {
    "dirichlet_ids": [1, 2, 3, 5, 6, 7],
    "diagnostic_solver_type": "icepack",
    "diagnostic_solver_kwargs": {
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}
solver = icepack.solvers.FlowSolver(model, **opts)

In [ ]:
u = solver.diagnostic_solve(
    velocity=u,
    thickness=h,
    surface=s,
    fluidity=A,
    friction=C,
)

The results are much faster than observations but not so far out there that we can't tweak them a bit.

In [ ]:
fig, axes = subplots()
streamlines = firedrake.streamplot(
    u, norm=log_norm, axes=axes, resolution=2.5e3, seed=1729
)
fig.colorbar(streamlines);